In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:22pt; line-height:40px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# <span style="color:rgb(0,200,100);">1. 패키지 </span>

In [3]:
# python-dotenv, langchain(1.2.0),  langchain-openai,  langchain-pinecone,
# pandas,  langchain-community, docx2txt,  langchain-text_splitters,  langchain_ollama
%pip install -q cohere

Note: you may need to restart the kernel to use updated packages.


# <span style="color:rgb(0,200,100);">2. 환경설정(환경변수, 시스템파라미터변수) </span>

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_LLM_MODEL = "gpt-4o-mini"
OPENAI_EMBEDDING_MODEL = "text-embedding-3-large" # 차원수 3072

PINECONE_INDEX_NAME = "better-rag-index"

# cohere 사이트 회원가입 후 API key받아 .env (COHERE_API_KEY) 커널 재시작 후 셀 실행
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

# <span style="color:rgb(0,200,100);">3. category추출하기 </span>
- chunk 카테고리, query 카테고리

In [2]:
def get_category():
    return {
        '납세의무': {
            '납세의무': 3, '거주자': 3, '비거주자': 3, '납세의무자': 3, 
            '원천징수': 2, '원천징수의무자': 2, '공동사업자': 2, 
            '상속인': 2, '증여자': 2, '신탁재산': 2
        },

        '세율계산': {
            '세율': 3, '소득세': 3, '과세표준': 3, '산출세액': 3, '세액': 3,
            '결정세액': 2, '세액계산': 2, '기본세율': 2, '세율적용': 2,
            '누진세율': 2, '종합소득세': 2
        },

        '근로소득': {
            '근로소득': 3, '총급여': 3, '급여': 3, '연봉': 3, '임금': 3,
            '근로소득금액': 2, '총급여액': 2, '상여': 2, '수당': 2,
            '봉급': 2, '직장인': 2
        },

        '사업소득': {
            '사업소득': 3, '총수입금액': 3, '필요경비': 3,
            '사업자': 2, '사업소득금액': 2, '결손금': 2, '이월결손금': 2,
            '주택임대소득': 2, '공동사업': 2
        },

        '이자배당': {
            '이자소득': 3, '배당소득': 3, '예금이자': 2, '채권': 2,
            '의제배당': 2, '배당세액공제': 2, '분리과세이자소득': 2,
            '분리과세배당소득': 2
        },

        '양도소득': {
            '양도소득': 3, '자산양도': 2, '부동산양도': 2, '주식양도': 2,
            '양도차익': 2, '취득가액': 2, '양도가액': 2, '양도소득금액': 2
        },

        '연금소득': {
            '연금소득': 3, '연금계좌': 3, '연금저축': 3,
            '퇴직연금': 2, '공적연금': 2, '사적연금': 2, '연금보험': 2,
            '연금수령': 2
        },

        '기타소득': {
            '기타소득': 3, '가상자산': 3, '가상자산소득': 2,
            '상금': 2, '보상금': 2, '종교인소득': 2, '원고료': 2,
            '복권': 1, '당첨금': 1, '발명보상금': 1
        },

        '공제감면': {
            '공제': 3, '소득공제': 3, '세액공제': 3,
            '기본공제': 2, '인적공제': 2, '특별공제': 2, '추가공제': 2,
            '근로소득공제': 2, '연금소득공제': 2, '퇴직소득공제': 2,
            '연금계좌세액공제': 2, '감면': 2
        },

        '비과세': {
            '비과세': 3, '비과세소득': 3, '면제': 2, '세액감면': 2,
            '소득세면제': 2, '복무급여': 1, '실업급여': 1, 
            '출산휴가급여': 1, '장학금': 1
        },

        '신고납부': {
            '신고': 3, '확정신고': 3, '과세표준확정신고': 3, 
            '납부': 3, '중간예납': 2, '가산세': 2,
            '신고기한': 2, '납부기한': 2, '납세지': 2
        },

        '과세기간': {
            '과세기간': 3, '과세연도': 3, '사업연도': 2, 
            '과세기간종료일': 2
        },

        '과세방식': {
            '종합과세': 3, '분리과세': 3, '합산과세': 2, 
            '종합소득과세표준': 2, '분리과세소득': 2, 
            '금융투자소득': 2
        },

        '장부기장': {
            '장부': 3, '복식부기': 3, '간편장부': 2, '기장': 2,
            '장부기록': 2, '증명서류': 2, '기장세액공제': 2
        }
    }

In [3]:
content = """1. 원천징수하는 자가 거주자인 경우: 그 거주자의 주된 사업장 소재지. 다만, 주된 사업장 외의 사업장에서 원천징수를 하는 경우에는 그 사업장의 소재지, 사업장이 없는 경우에는 그 거주자의 주소지 또는 거소지로 한다.
2. 원천징수하는 자가 비거주자인 경우: 그 비거주자의 주된 국내사업장 소재지. 다만, 주된 국내사업장 외의 국내사업장에서 원천징수를 하는 경우에는 그 국내사업장의 소재지, 국내사업장이 없는 경우에는 그 비거주자의 거류지(居留地) 또는 체류지로 한다.
3. 장부 장부 장부 장부 복식부기 복식부기 복식부기 복식부기 복식부기 복식부기
"""

In [5]:
def categorize_content(content, top_k=None):
    """내용 카테고리 분류 - 점수 기반으로 모든 카테고리를 점수 순으로 반환
    Parameters :
    - content : 분류할 텍스트 내용
    - top_k : 상위 몇개까지 카테고리를 반환할지(None이면 모든 카테고리 반환)
    Returns:
    - 카테고리 리스트(점수 높은 순)"""
    category_keywords = get_category()
    category_scores = {}
    # 각 카테고리별 점수 계산
    for category, weighted_keywords in category_keywords.items():
        # print(category, weighted_keywords)
        score = 0
        for keyword, weight in weighted_keywords.items():
            #if keyword in content: # content에 keyword 가 포함되어 있는지 여부
            #    score += weight
            count = content.count(keyword) # content에 keyword가 몇번 나오는지
            score += count * weight
        if score > 0:
            category_scores[category] = score
    # print(category_scores)
    # 내림차순 정렬한 카테고리 이름만 추출
    sorted_categories = sorted(category_scores.items(), key=lambda x:x[1], reverse=True) # 내림차순
    #print(sorted_categories)
    all_categories = [category[0] for category in sorted_categories]
    # print(all_categories)
    # 매칭되는 카테고리가 없으면 "기타" 반환
    if not all_categories:
        all_categories = ["기타"]
    # top_k가 지정되면 상위 top_k개만, 아니면 전체 반환
    if top_k is not None:
        return all_categories[:top_k]
    else:
        return all_categories
type(categorize_content(content))

list

# <span style="color:rgb(0,200,100);">4. 임베딩 모델 설정 </span>

In [6]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(
    model=OPENAI_EMBEDDING_MODEL,
    openai_api_key=OPENAI_API_KEY
)

# <span style="color:rgb(0,200,100);">5. Pinecone vector store 가져오기 </span>

In [7]:
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
# pinecone 클라언트
pc = Pinecone()
vector_database = PineconeVectorStore(
    embedding=embedding,
    index_name=PINECONE_INDEX_NAME
)

# <span style="color:rgb(0,200,100);">6. 키워드 사전 활용하여 질문 표준화 </span>

In [10]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
llm = ChatOpenAI(model=OPENAI_LLM_MODEL)
keyword_dict  = [
    "사람을 나타내는 표현 -> 거주자",
    "직장인 -> 근로소득이 있는 거주자", 
    "월급쟁이 -> 근로소득이 있는 거주자",
    "회사원 -> 근로소득이 있는 거주자",
    "연봉 -> 종합소득",
    "월급 -> 근로소득",
    "세금 -> 소득세",
    "공제받다 -> 공제를 적용받다",
    "얼마나 내야하나 -> 세액은 얼마인가",
    "계산해줘 -> 계산하면 얼마인가"
]
prompt = ChatPromptTemplate.from_template(f"""사용자의 질문을 보고, 우리의 사전을 참고해서 
사용자의 질문을 변경해 주세요. 만약 변경할 필요가 없을경우, 사용자의 질문을 변경하지 않아도 됩니다.
그런 경우에는 질문만 리턴해 주세요.
사전 : {keyword_dict}
질문 : {{question}}""")
keyword_chain = prompt | llm | StrOutputParser()
query = "연봉 5000만원인 직장인의 소득세는 얼마예요?"
normalized_query = keyword_chain.invoke({"question":query})
normalized_query

'연봉 5000만원인 근로소득이 있는 거주자의 소득세는 세액은 얼마인가?'

# <span style="color:rgb(0,200,100);">7. 관련문서 미리보기(category => title과 text에 의해 rerank) </span>

In [11]:
categorize_content(normalized_query)

['세율계산', '근로소득', '납세의무']

In [14]:
# Retriever 생성 (category중 하나라도 포함되는 있는 10개 chunk 추출)
retriever = vector_database.as_retriever(
    search_kwargs = {
        "k":10,
        "filter":{"category":{"$in":categorize_content(normalized_query)}}
    }
)
docs = retriever.invoke(normalized_query)
print("관련 문서")
for i, doc in enumerate(docs):
    chunk_id = doc.metadata.get('chunk_id')
    title = doc.metadata.get('title')
    print(f"{i+1}.{chunk_id}- {title}")

관련 문서
1.chunk_046- 제55조세율 거주자의 종합소득에 대한 소득세 세율 및 퇴직소득 산출세액 계산 방법.
2.chunk_038- 소득세법 제46조 및 제46조의2의 핵심 제목: 채권 등에 대한 소득금액의 계산 특례 및 중도 해지로 인한 이자소득금액 계산의 특례
3.chunk_005- 소득세법 제10조납세지의 변경신고 및 제11조과세 관할와 제12조비과세소득 관련 규정 요약.
4.chunk_058- 소득세법 제1항부터 제4항까지의 공제 신청, 교육비 세액공제 확인을 위한 자료 요청, 2024년 기부금 공제 및 추가 공제, 표준세액공제 규정, 특별세액공제 정의 및 관련 사항, 종합소득금액의 세액 감면 규정.
5.chunk_039- 제5관 근로소득공제, 연금소득공제 및 퇴직소득공제
6.chunk_002- 공동 소유 자산의 양도소득금액 계산 및 납세의무
7.chunk_045- 소득세법 제54조 및 제54조의2의 주요 내용은 종합소득공제의 배제와 공동사업에 대한 소득공제 특례에 관한 규정으로, 특정 소득에 대한 공제를 적용하지 않거나, 공동사업자의 소득에 대한 합산과세 시 소득공제를 받을 수 있는 조건을 명시하고 있습니다.
8.chunk_040- 퇴직소득공제 및 기본공제, 추가공제에 관한 규정
9.chunk_009- 소득세법 제14조: 과세표준의 계산 및 종합소득과세표준의 정의, 소득의 합산 제외 항목.
10.chunk_000- 소득세법의 핵심 제목: 소득세법의 목적, 정의 및 납세의무


### title+text 기반의 rerank함수

In [ ]:
import cohere
import numpy as np
def rerank_by_title(query:str, documents:list, top_k:int=4) -> list:
    """
    cohere의 rerank API를 하용한 문서 재정렬
    """

# <span style="color:rgb(0,200,100);">8. langchain 답변 생성 </span>

In [38]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

In [39]:
# 1. LLM과 임베딩 초기화
load_dotenv()
llm = ChatOpenAI(model = OPENAI_LLM_MODEL)
embedding = OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL)
# 2. 업로드한 벡터 db를 가져올 때
vector_database = PineconeVectorStore(
    embedding=embedding,
    index_name=PINECONE_INDEX_NAME
)

In [42]:
# 3. 키워드 사전 활용
keyword_dict  = [
    "사람을 나타내는 표현 -> 거주자",
    "직장인 -> 근로소득이 있는 거주자", 
    "월급쟁이 -> 근로소득이 있는 거주자",
    "회사원 -> 근로소득이 있는 거주자",
    "연봉 -> 종합소득",
    "월급 -> 근로소득",
    "세금 -> 소득세",
    "공제받다 -> 공제를 적용받다",
    "얼마나 내야하나 -> 세액은 얼마인가",
    "계산해줘 -> 계산하면 얼마인가"
]
prompt = ChatPromptTemplate.from_template(f"""사용자의 질문을 보고, 우리의 사전을 참고해서 
사용자의 질문을 변경해 주세요. 만약 변경할 필요가 없을경우, 사용자의 질문을 변경하지 않아도 됩니다.
그런 경우에는 질문만 리턴해 주세요.
사전 : {keyword_dict}
질문 : {{question}}""")
# llm.invoke(prompt.invoke("연봉 5000만원인 직장인의 소득세는 얼마예요?"))
keyword_chain = prompt | llm | StrOutputParser()
keyword_chain.invoke({"question":"연봉 5000만원인 직장인의 소득세는 얼마예요?"})

'종합소득 5000만원인 근로소득이 있는 거주자의 소득세는 세액은 얼마인가?'

In [43]:
keyword_chain.invoke("월급이 높은 남자가 있습니다")

'근로소득이 있는 거주자가 있습니다.'

In [45]:
# 3. Retriever 생성(category중 하나라도 포함되는 있는 chunk만)
retriever = vector_database.as_retriever(
    search_kwargs = {
        "k":3,
        "filter":{"category":{"$in":categorize_content(query)}}
    }
)
# 4. 프롬프트 템플릿
template = f"""당신은 최고의 한국 소득세 전문가입니다.
다음 문맥을 참고하여 질문에 답하세요.
답을 모르면 모른다고 답하세요.
최대 3문장으로 간결하게 답변하세요.
질문 : {{query}}
문맥 : {{context}}
답변 : """
prompt = ChatPromptTemplate.from_template(template)
# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return  "\n\n---\n\n".join([doc.page_content for doc in documents])
# 6. RAG 체인 구성 (LCEL 방식)
from langchain_core.runnables import RunnablePassthrough # {"query":"~"} => "~"
rag_chain = (
    {
        "context":retriever | format_documents,
        "query":RunnablePassthrough() # 질문 그대로 전달
    }
    | prompt # prompt에 context와 query 변수 주입
    | llm    
    | StrOutputParser()
)
final_chain = keyword_chain | rag_chain
# 7. 실행
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
result = final_chain.invoke(query)
print(result + "\n*위의 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 발생할 수 있습니다*")

종합소득 5천만원인 근로소득이 있는 거주자의 소득세는 84만원 + (1,400만원을 초과하는 금액 3,600만원의 15%)로 계산됩니다. 따라서, 소득세는 84만원 + 540만원 = 624만원입니다. 최종적으로, 해당 근로소득자의 소득세는 624만원입니다.
*위의 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 발생할 수 있습니다*


# <span style="color:rgb(0,200,100);">9. 참조 조항 포함한 답변 생성 </span>

In [47]:
# 3. Retriever 생성(category중 하나라도 포함되는 있는 chunk만)
query = "종합소득 5000만원인 근로소득이 있는 거주자의 소득세는 세액은 얼마인가?"
retriever = vector_database.as_retriever(
    search_kwargs = {
        "k":3,
        "filter":{"category":{"$in":categorize_content(query)}}
    }
)
documents = retriever.invoke(query)

In [60]:
# 참조조항을 추출하는 함수
def extract_articles_from_docs(documents):
    """검색된 문서들(documents)에서 article정보를 추출하여 리스로 변환
    Returns : 중복이 제거된 조항 리스트 ex : ['제4조', '제16조']
    """
    articles = []
    for doc in documents:
        article = doc.metadata.get('article', '조항없음')
        # 「」제거. 문자를 list로 분리
        if article != '조항없음':
            article = article.replace("「", "").replace("」", "")
            article_list = article.split(", ")
            articles.extend(article_list)
    # 중복 제거 및 정렬
    unique_articles = list(set(articles))
    unique_articles.sort(key=lambda x : int(x[:-1]))
    final_articles = ["제"+article for article in unique_articles]
    return "소득세법 " + ",".join(final_articles)
extract_articles_from_docs(documents)

'소득세법 제4조,제6조,제9조,제10조,제11조,제12조,제14조,제16조,제17조,제45조,제46조,제47조,제55조,제56조,제99조,제133조,제156조'

In [ ]:
# RAG 체인 구성 - 참조조항을 포함한 답변 출력

In [63]:
# 1. 질문을 표준화
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?" # 출력함
normalized_query = keyword_chain.invoke({"question":query}) # 출력함
# 2. 관련문서 검색
retrieved_docs = retriever.invoke(normalized_query)
referenced_articles = extract_articles_from_docs(retrieved_docs) # 참조조항 출력

# 프롬프트 템플릿(위에것 그대로)을 이용한 rag 체임 구성
template = f"""당신은 최고의 한국 소득세 전문가입니다.
다음 문맥을 참고하여 질문에 답하세요.
답을 모르면 모른다고 답하세요.
최대 3문장으로 간결하게 답변하세요.
질문 : {{query}}
문맥 : {{context}}
답변 : """
prompt = ChatPromptTemplate.from_template(template)
prompt_chain = prompt | llm | StrOutputParser()
result = prompt_chain.invoke({
    "context":format_documents(retrieved_docs), 
    "query":query
})

In [68]:
print("원본 질문 :", {query})
print("표준화된 질문 :", {normalized_query})
print("="*70)
print("\n✅ 답변 :", result)
print("\n📌 참조조항 :", referenced_articles)
print("\n* 위 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 날 수 있습니다 *")

원본 질문 : {'연봉 5천만원인 직장인의 소득세는 얼마인가요?'}
표준화된 질문 : {'연봉 5천만원인 근로소득이 있는 거주자의 소득세는 얼마인가요?'}

✅ 답변 : 연봉 5천만원인 직장인의 소득세는 84만원에 1,400만원을 초과하는 금액의 15%를 더한 금액으로 계산됩니다. 즉, 5천만원에서 1,400만원을 빼면 3,600만원이 남으며, 이 금액에 15%를 적용하면 540만원이 추가됩니다. 따라서 총 소득세는 84만원 + 540만원 = 624만원입니다.

📌 참조조항 : 소득세법 제4조,제6조,제9조,제10조,제11조,제12조,제14조,제16조,제17조,제45조,제46조,제47조,제55조,제56조,제99조,제133조,제156조

* 위 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 날 수 있습니다 *


In [70]:
# 자사용 가능한 함수로 만들기
def ask_with_reference(query:str, k:int=4):
    "query, 표준화query, 답변, 참조사항 출력"
    # llm, embedding, vector_database, keyword_chain
    # 1. 질문을 표준화
    normalized_query = keyword_chain.invoke({"question":query}) # 출력함
    # 2. 관련문서 검색
    retriever = vector_database.as_retriever(
        search_kwargs = {
            "k":k,
            "filter":{"category":{"$in":categorize_content(query)}}
        }
    )
    # 관련문서 rerank
    retrieved_docs = retriever.invoke(normalized_query)
    referenced_articles = extract_articles_from_docs(retrieved_docs) # 참조조항 출력

    # 프롬프트 템플릿(위에것 그대로)을 이용한 rag 체임 구성
    template = f"""당신은 최고의 한국 소득세 전문가입니다.
    다음 문맥을 참고하여 질문에 답하세요.
    답을 모르면 모른다고 답하세요.
    최대 3문장으로 간결하게 답변하세요.
    질문 : {{query}}
    문맥 : {{context}}
    답변 : """
    prompt = ChatPromptTemplate.from_template(template)
    prompt_chain = prompt | llm | StrOutputParser()
    result = prompt_chain.invoke({
        "context":format_documents(retrieved_docs), 
        "query":query
    })
    print("원본 질문 :", {query})
    print("표준화된 질문 :", {normalized_query})
    print("="*70)
    print("\n✅ 답변 :", result)
    print("\n📌 참조조항 :", referenced_articles)
    print("\n* 위 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 날 수 있습니다 *")

In [71]:
# 사용 예시
ask_with_reference("연봉 5천만원인 직장인 소득세는?")

원본 질문 : {'연봉 5천만원인 직장인 소득세는?'}
표준화된 질문 : {'종합소득 5천만원인 근로소득이 있는 거주자 소득세는?'}

✅ 답변 : 연봉 5천만원인 직장인의 소득세는 84만원에 1,400만원을 초과하는 금액에 대해 15%를 적용하여 계산합니다. 구체적으로, 5천만원에서 1,400만원을 빼면 3,600만원이고, 이 금액의 15%는 540만원입니다. 따라서 소득세는 84만원 + 540만원 = 624만원이 됩니다.

📌 참조조항 : 소득세법 제4조,제6조,제9조,제10조,제11조,제12조,제14조,제16조,제17조,제22조,제45조,제46조,제47조,제48조,제49조,제55조,제56조,제99조,제133조,제156조

* 위 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 날 수 있습니다 *


In [72]:
# 사용 예시
ask_with_reference("양도 소득세는 얼마인가요?")

원본 질문 : {'양도 소득세는 얼마인가요?'}
표준화된 질문 : {'양도 소득세는 얼마인가요?'}

✅ 답변 : 양도소득세는 거주자의 양도소득 과세표준에 따라 세율을 적용하여 계산됩니다. 세율은 자산의 종류 및 보유 기간에 따라 다르며, 일반적으로 6%부터 시작하여 최대 45%까지 적용될 수 있습니다. 구체적인 세율과 계산 방식은 양도소득에 대한 법률에 명시되어 있습니다.

📌 참조조항 : 소득세법 제14조,제17조,제47조,제55조,제56조,제90조,제93조,제94조,제95조,제97조,제101조,제102조,제103조,제104조,제114조,제115조,제283조,제286조,제360조

* 위 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 날 수 있습니다 *
